In [1]:
from pyomo.environ import *
import os
model = AbstractModel()
solverexe = "gurobi"
dirsolver = r"C:\Users\ch9fod\Documents\GitHub\ED\solvers"
datafile = "data0.dat"

In [2]:
#set
model.G = Set()

In [3]:
#parameters
model.a = Param(model.G)
model.b = Param(model.G)
#lone parameter
model.D = Param()

In [4]:
#variables
model.P = Var(model.G)

In [5]:
#constraints
def demand_r(model,i):
    return model.D == sum(model.P[i] for i in model.G)
model.demand = Constraint(model.G, rule = demand_r)

In [6]:
#objective
def cost_rule(model):
    return sum(model.a[i]*model.P[i] + 
               0.5*model.b[i]*model.P[i]**2 for i in model.G)
#default is to minimize        
model.OBJ = Objective(rule=cost_rule) 

In [7]:
if solverexe == "gurobi":
    solver = SolverFactory(solverexe)
else:
    solver = SolverFactory(solverexe, 
                           executable=os.path.join(dirsolver, solverexe))
instance = model.create_instance(datafile)
instance.dual = Suffix(direction=Suffix.IMPORT)
results = solver.solve(instance)
# results = solver.solve(instance, tee=True)
#instance.solutions.load_from(results)
#print(results)

In [8]:
instance.display()
# model.solutions.load_from(results)

Model unknown

  Variables:
    P : Size=3, Index=G
        Key : Lower : Value         : Upper : Fixed : Stale : Domain
          1 :  None : 428.571428571 :  None : False : False :  Reals
          2 :  None : 164.285714286 :  None : False : False :  Reals
          3 :  None : 7.14285714295 :  None : False : False :  Reals

  Objectives:
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 18910.714285712216

  Constraints:
    demand : Size=3
        Key : Lower : Body            : Upper
          1 :   600 : 599.99999999995 :   600
          2 :   600 : 599.99999999995 :   600
          3 :   600 : 599.99999999995 :   600


In [9]:
print ("Duals")
from pyomo.core import Constraint
for c in instance.component_objects(Constraint, active=True):
    print ("   Constraint",c)
    cobject = getattr(instance, str(c))
    for index in cobject:
        print ("      ", index, cobject, instance.dual[cobject[index]])

Duals
   Constraint demand
       1 demand 41.4285714285
       2 demand -0.0
       3 demand -0.0
